# **Text Gender Classification**

## Load Dataset

In [1]:
!chmod 600 /content/kaggle.json

In [2]:
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d surekharamireddy/customer-reviews-on-second-hand-clothing-stores

  0% 0.00/444k [00:00<?, ?B/s]
100% 444k/444k [00:00<00:00, 81.2MB/s]


In [3]:
!mkdir /content/Data

In [4]:
import zipfile
zip_file = zipfile.ZipFile('/content/customer-reviews-on-second-hand-clothing-stores.zip')
zip_file.extractall('/content/Data')

In [8]:
import pandas as pd
import numpy as np

DATA = pd.read_csv("/content/Data/JAM_Reviews_En.csv")
DATA.head()

,revUserFace href,revUserFace src,revUserFaceName 2,Author_En,revUserFaceDtlTxt,Generation _En,Gender_En,revUserFaceDtlTxt 2,revUserFaceDtlTxt 3,revUserFaceDtlTxt 4,...,revUserDispList 2,revDispListNum 2,revUserDispList 3,revDispListNum 3,revUserDispList 4,revDispListNum 4,revUserDispList 5,revDispListNum 5,revUserDispList 6,revDispListNum 6
0,https://review.rakuten.co.jp/my/ODhjZGVlZjI0OW...,https://image.space.rakuten.co.jp/d/strg/ctrl/...,shibanii0209,shibanii0209,50代 男性,50,male,購入者,レビュー投稿,件,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://review.rakuten.co.jp/my/MTliMTQ1ZjFkY2...,https://image.space.rakuten.co.jp/d/strg/ctrl/...,二郎123,123,購入者,購入,NaN,レビュー投稿,件,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://review.rakuten.co.jp/my/ODNlZjM2YWUzZG...,https://image.space.rakuten.co.jp/d/strg/ctrl/...,ｍ６５,M65,50代 男性,50,male,購入者,レビュー投稿,件,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,https://image.space.rakuten.co.jp/d/strg/ctrl/...,購入者,buyer,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://review.rakuten.co.jp/my/YTI4OGU5MzNmYj...,https://image.space.rakuten.co.jp/d/strg/ctrl/...,dyezine,DYEZINE,40代 男性,40,male,購入者,レビュー投稿,件,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Filtering

In [10]:
# Menghilangkan data yang terdapat NaN
DATADROP = DATA.dropna()
data = DATADROP['Review_En']
df = pd.DataFrame(list(data), columns = ['Review'])
df.head()

,Review
0,"The telephone correspondence was also good, an..."
1,"I have a lot of things, so I won't buy it when..."
2,Thank you for your prompt response.
3,"Quick, polite message\nProducts were also very..."
4,Thank you for your prompt and courteous respon...


In [11]:
# Mengubah menjadi lowercase
data = data.str.lower()

print('Case Folding Result :\n')
df = pd.DataFrame(list(data), columns = ['Review'])
df.head()

Case Folding Result :



,Review
0,"the telephone correspondence was also good, an..."
1,"i have a lot of things, so i won't buy it when..."
2,thank you for your prompt response.
3,"quick, polite message\nproducts were also very..."
4,thank you for your prompt and courteous respon...


In [115]:
# Menghilangkah karakter tertentu
import string 
import re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def remove_special(text):
    text = text.replace('\\t',' ').replace('\\n'," ").replace('\\u'," ").replace('\\'," ")    # menghilangkan whitespace
    text = text.encode('ascii','replace').decode('ascii')                                     # convert to ascii
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)|(\(\W+\))"," ", text).split())   # remove hashtag,tag,url
    text = ' '.join(re.sub(r"\d+","",text).split())                                           # remove digit
    text = ' '.join(re.sub('\s+',' ',text).split())                                           # remove multiple whitespace
    text = text.translate(str.maketrans("","",string.punctuation))                            # remove Punctuation
    text = text.strip()                                                                       # remove whitespace di awal dan akhir kalimat
    return text.replace("http://"," ").replace("https://", " ")

data = data.apply(remove_special)
df = pd.DataFrame(list(data), columns = ['Review'])
df.head()

,Review
0,the telephone correspondence was also good and i had a good shopping shopping thank you very much
1,i have a lot of things so i wont buy it when buying
2,thank you for your prompt response
3,quick polite message products were also very beautiful because it is a refreshing color wearing im fine thank you very much
4,thank you for your prompt and courteous response we will also use it


In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [58]:
# Tokenization
def word_tokenize_wrapper(text):
  return word_tokenize(text)

data_tokens = data.apply(word_tokenize_wrapper)
print('Tokenizing Result : \n')

# Tampilkan Tabular
list_ = list(zip(data_tokens))
df_tokens = pd.DataFrame(list_, columns = ['Review'])
df_tokens.head()

Tokenizing Result : 



,Review
0,"[the, telephone, correspondence, was, also, go..."
1,"[i, have, a, lot, of, things, so, i, wont, buy..."
2,"[thank, you, for, your, prompt, response]"
3,"[quick, polite, message, products, were, also,..."
4,"[thank, you, for, your, prompt, and, courteous..."


In [59]:
# Frequency Distribution
def freqDist_wrapper(text):
  return FreqDist(text)

data_fdist = data_tokens.apply(freqDist_wrapper)
print('Frequency Tokens : \n')

# Tampilkan Tabular
list_ = list(zip(data_fdist))
df_tokens = pd.DataFrame(list_, columns = ['Review'])
df_tokens.head()

Frequency Tokens : 



,Review
0,"{'the': 1, 'telephone': 1, 'correspondence': 1..."
1,"{'i': 2, 'have': 1, 'a': 1, 'lot': 1, 'of': 1,..."
2,"{'thank': 1, 'you': 1, 'for': 1, 'your': 1, 'p..."
3,"{'quick': 1, 'polite': 1, 'message': 1, 'produ..."
4,"{'thank': 1, 'you': 1, 'for': 1, 'your': 1, 'p..."


In [37]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
from nltk.corpus import stopwords
list_stopwords = stopwords.words('english')
len(list_stopwords)

179

In [85]:
list_stopwords = set(list_stopwords)

def stopwords_removal(words):
  return [word for word in words if word not in list_stopwords]

tokens_wsw = data_tokens.apply(stopwords_removal)

data_wsw = [' '.join(a) for a in tokens_wsw]
data_wsw = pd.DataFrame(data_wsw,columns = ['Review'])

# Tampilkan Tabular
list_ = list(zip(tokens_wsw))
df_tokens = pd.DataFrame(list_, columns = ['Review'])
df_tokens.head()

,Review
0,"[telephone, correspondence, also, good, good, ..."
1,"[lot, things, wont, buy, buying]"
2,"[thank, prompt, response]"
3,"[quick, polite, message, products, also, beaut..."
4,"[thank, prompt, courteous, response, also, use]"


In [61]:
# Cari Freq Distribution dari Data tanpa stop words
tokens_wsw_fdist = tokens_wsw.apply(freqDist_wrapper)
data_clean = tokens_wsw_fdist.apply(lambda x : x.most_common())

# Tampilkan Tabular
list_ = list(zip(data_clean))
df_tokens = pd.DataFrame(list_, columns = ['Review'])
df_tokens.head()

,Review
0,"[(good, 2), (shopping, 2), (telephone, 1), (co..."
1,"[(lot, 1), (things, 1), (wont, 1), (buy, 1), (..."
2,"[(thank, 1), (prompt, 1), (response, 1)]"
3,"[(quick, 1), (polite, 1), (message, 1), (produ..."
4,"[(thank, 1), (prompt, 1), (courteous, 1), (res..."


## Feature Extraction

In [94]:
DATADROP.Gender_En.value_counts()

male     19
woman    12
Name: Gender_En, dtype: int64

In [52]:
DATADROP.shape

(31, 51)

In [92]:
# Ambil data Riview yang telah dihilangkan Stop Words nya
clean = data_wsw['Review']

In [91]:
# Vectorizing dengan TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 1000)
vectors = vectorizer.fit_transform(clean)

words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,accompany,actual,already,also,always,answered,anxiety,arrived,assortment,...,wants,wear,wearing,within,without,wont,word,would,years,yesterday
0,0.0,0.0,0.0,0.0,0.192667,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.510795,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.167709,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.33286,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.305465,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Classification

### Train Model

In [95]:
# Split Feature dengan Label nya
x = words_df
y = DATADROP.Gender_En

In [96]:
# Train Model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

logreg = LogisticRegression(C=1e9, solver='lbfgs', max_iter=1000)
logreg.fit(x, y)
forest = RandomForestClassifier(n_estimators=50)
forest.fit(x,y)
svc = LinearSVC()
svc.fit(x,y)
bayes = MultinomialNB()
bayes.fit(x,y)

MultinomialNB()

### Test Model

In [111]:
# Buat dataframe yang ingin di klasifikasikan
pd.set_option("display.max_colwidth", 200)

unknown = pd.DataFrame({'content':["Well Bill nobody will be able to say that you and the New York Times did warn us",
                                   "I applaud the spirit of this column and agree one should approach politics",
                                   "I found myself immensely enjoying this when I went to see it",
                                   "I had like to see the discussion moved up a level in exchange for a corp",
                                   "Using Chinese basic sentiment words they can identify sentiment words and expand their sentiment lexicon",
                                   "we prepared two lists containing sentiment words in Hebrew",
                                   "The battery life of this camera is too short"]})
unknown

,content
0,Well Bill nobody will be able to say that you and the New York Times did warn us
1,I applaud the spirit of this column and agree one should approach politics
2,I found myself immensely enjoying this when I went to see it
3,I had like to see the discussion moved up a level in exchange for a corp
4,Using Chinese basic sentiment words they can identify sentiment words and expand their sentiment lexicon
5,we prepared two lists containing sentiment words in Hebrew
6,The battery life of this camera is too short


In [112]:
# Feature Extract
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,accompany,actual,already,also,always,answered,anxiety,arrived,assortment,...,wants,wear,wearing,within,without,wont,word,would,years,yesterday
0,0.533709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
# Prediksi dengan model classifier

# Logistic Regression prediction dan probabilities
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
unknown['logreg_prob'] = logreg.predict_proba(unknown_words_df)[:,1]

# Random Forest prediction dan probabilities
unknown['pred_forest'] = forest.predict(unknown_words_df)
unknown['forest_prob'] = forest.predict_proba(unknown_words_df)[:,1]

# SVC prediction
unknown['pred_svc'] = svc.predict(unknown_words_df)

# Bayes prediction dan probabilities
unknown['pred_bayes'] = bayes.predict(unknown_words_df)
unknown['bayes_prob'] = bayes.predict_proba(unknown_words_df)[:,1]

unknown

,content,pred_logreg,logreg_prob,pred_forest,forest_prob,pred_svc,pred_bayes,bayes_prob
0,Well Bill nobody will be able to say that you and the New York Times did warn us,woman,0.714435,male,0.24,male,male,0.454774
1,I applaud the spirit of this column and agree one should approach politics,male,0.027559,male,0.16,male,male,0.387097
2,I found myself immensely enjoying this when I went to see it,male,0.027559,male,0.16,male,male,0.387097
3,I had like to see the discussion moved up a level in exchange for a corp,woman,0.995983,male,0.38,woman,male,0.497003
4,Using Chinese basic sentiment words they can identify sentiment words and expand their sentiment lexicon,male,0.027559,male,0.16,male,male,0.387097
5,we prepared two lists containing sentiment words in Hebrew,male,0.027559,male,0.16,male,male,0.387097
6,The battery life of this camera is too short,male,0.027559,male,0.16,male,male,0.387097
